# Fine-tuning Llama

Note that this notebook is created based on the following [link](https://github.com/teddylee777/langchain-kr/tree/main/18-FineTuning) and [unsloth colab](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing).

In [1]:
import torch

# Get the major and minor version of the CUDA device..
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 9)

# Describes the process of installing the `unsloth` library and its dependencies.

In [2]:
import torch
torch.__version__

'2.7.1+cu128'

In [ ]:
%%capture
!uv pip install "unsloth[cu128-ampere-torch270] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # New GPU(e.g., Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)
    !uv pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Old GPU(e.g., V100, Tesla T4, RTX 20xx)
    !uv pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

# Fine-tuning with Unsloth
![](https://docs.unsloth.ai/~gitbook/image?url=https%3A%2F%2F2815821428-files.gitbook.io%2F%7E%2Ffiles%2Fv0%2Fb%2Fgitbook-x-prod.appspot.com%2Fo%2Forganizations%252FHpyELzcNe0topgVLGCZY%252Fsites%252Fsite_mXXTe%252Flogo%252Fqb5xJcZ4NHJqPEnbI3U2%252Funsloth%2520final%2520space%2520white.png%3Falt%3Dmedia%26token%3Ded5bbc94-79d4-4abd-8b76-5d7ab3a1d468&width=260&dpr=1&quality=100&sign=fb146c36&sv=2)

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jake/git/vllm-chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-08 19:03:40 [__init__.py:235] Automatically detected platform cuda.
==((====))==  Unsloth 2025.8.2: Fast Llama patching. Transformers: 4.55.0. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4060 Ti. Num GPUs = 1. Max memory: 15.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,   # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    lora_alpha=32,  
    lora_dropout=0.05,   # Supports any, but = 0 is optimized
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  
    bias="none",  # 바이어스를 지원합니다.
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=123,  
    use_rslora=False,   # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.8.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


# Data Prep

In [9]:
from datasets import load_dataset

# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""


# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["question"]  # 지시사항을 가져옵니다.
    outputs = examples["answer"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


# "teddylee777/QA-Dataset-mini" 데이터셋을 불러옵니다. 훈련 데이터만 사용합니다.
dataset = load_dataset("Jake5/movensys-info", split="train")

# 데이터셋에 formatting_prompts_func 함수를 적용합니다. 배치 처리를 활성화합니다.
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map: 100%|██████████| 23/23 [00:00<00:00, 6748.44 examples/s]


# Training

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=dataset,  # 학습 데이터셋
    eval_dataset=dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이
    dataset_num_proc=2,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        max_steps=100,  # 최대 스텝 수
        do_eval=True,
        # evaluation_strategy="steps",
        logging_steps=1,  # logging 스텝 수
        learning_rate=2e-4,  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=123,  # 랜덤 시드
        output_dir="outputs",  # 출력 디렉토리
    ),
)

Unsloth: Tokenizing ["text"]: 100%|██████████| 23/23 [00:00<00:00, 5398.98 examples/s]


In [12]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA GeForce RTX 4060 Ti. Max memory = 15.996 GB.
7.088 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23 | Num Epochs = 34 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.312100
2,3.306500
3,3.053400
4,2.913800
5,3.112700
6,2.466700
7,2.088200
8,2.007800
9,2.209500
10,1.774000


In [14]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

290.2466 seconds used for training.
4.84 minutes used for training.
Peak reserved memory = 7.447 GB.
Peak reserved memory for training = 0.359 GB.
Peak reserved memory % of max memory = 46.555 %.
Peak reserved memory for training % of max memory = 2.244 %.


In [15]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [21]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            # "Tell me about movensys company",  # 지시사항
            # "what's movensys wmx?",  # 지시사항
            # "what's main product of movensys?",  # 지시사항
            "what's relationship between movensys and skylla?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
what's relationship between movensys and skylla?

### Response:
Movensys is a member of the Skylla family, specifically a subsidiary. Its parent company is Skylla Technologies, and it is also part of the Skylla ecosystem which includes Skylla Galaxy, Skylla Nova, and Skylla Jupiter.<|end_of_text|>


In [16]:
model.save_pretrained("Meta-Llama-3.1-8B-wmx")  # 모델을 로컬에 저장합니다.
# model.push_to_hub("your_name/lora_model", token = "...") # 모델을 온라인 허브에 저장합니다.